In [1]:
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as yh
from stock_clustering import clusters_dict
from stock_dataset import stock_data

import warnings

warnings.filterwarnings("ignore")

/Users/miladziekanowska/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Stock data ready to use!
Clustering done!


In [3]:
clusters_dict[2] = clusters_dict.pop(4)
clusters_dict.items()

dict_items([(0, ['AXP', 'BA', 'CSCO', 'DOW', 'GS', 'HD', 'HON', 'JPM', 'NKE', 'V']), (1, ['CAT', 'CVX', 'IBM', 'JNJ', 'KO', 'MCD', 'MRK', 'TRV', 'UNH']), (3, ['CRM', 'DIS', 'INTC', 'MMM', 'VZ', 'WBA']), (2, ['AMGN'])])

Teraz co ja chcę z tym zrobić?
- potrzebuję utworzyć osobne słowniki dfów dla każdego klastra - ZROBIONE
- ekperymentalnie, na jednym ze słowników zrobić dwa modele - dla close i dla daily change:
    - najpierw zmienić dane na windowed df dla lstm'a (długość? trzy ostatnie dni starczą chyba);
    - rozdzielić na train val test
    - zbudować model dla close i model dla dailychange
    - porównać modele pod kątem sprawdzania się dla różnych firm
    - jeśli źle, przemyśleć klastryzację
    - jeśli w miarę dobrze, wybrać na czym opieramy model (close czy dailychange)
- stworzyć modele dla wszystkich klastrów
- ocenić ich jakość
- zapisać modele
- sprawdzić, jak je douczać

In [7]:
stock_data['AAPL'].shape

(753, 2)

In [4]:
cluster_data = {}
for cluster, names in clusters_dict.items():
    cluster_data[cluster] = {name: stock_data[name] for name in names}

In [5]:
cluster_0 = {}
for name, df in cluster_data[0].items():
    cluster_0[name] = df
    
cluster_1 = {}
for name, df in cluster_data[1].items():
    cluster_1[name] = df
    
cluster_2 = {}
for name, df in cluster_data[2].items():
    cluster_2[name] = df

cluster_3 = {}
for name, df in cluster_data[3].items():
    cluster_3[name] = df

In [7]:
cluster_0['CSCO']

array([[-0.39092168, -0.01458432],
       [-0.38223968,  0.05461382],
       [-0.40307661, -0.18048431],
       ...,
       [ 0.40088827,  0.99752315],
       [ 0.20119965, -1.46532054],
       [ 0.27586603,  0.54024857]])

In [8]:
closing = {}
for key in cluster_0.keys():
    closing[key] = cluster_0[key][:,0]

In [10]:
daily_change = {}
for key in cluster_0.keys():
    daily_change[key] = cluster_0[key][:,1]

In [19]:
window = 4

X = {}
Y = {}

for key in closing.keys():
    X[key] = []
    Y[key] = []
    for i in range(window, len(closing[key])):
        X[key].append(closing[key][i-window:i])
        Y[key].append(closing[key][i])

[array([-0.39092168, -0.38223968, -0.40307661, -0.31451916]),
 array([-0.38223968, -0.40307661, -0.31451916, -0.48989793]),
 array([-0.40307661, -0.31451916, -0.48989793, -0.35272075]),
 array([-0.31451916, -0.48989793, -0.35272075, -0.51073552]),
 array([-0.48989793, -0.35272075, -0.51073552, -0.43606914]),
 array([-0.35272075, -0.51073552, -0.43606914, -0.48295272]),
 array([-0.51073552, -0.43606914, -0.48295272, -0.36834862]),
 array([-0.43606914, -0.48295272, -0.36834862, -0.35792982]),
 array([-0.48295272, -0.36834862, -0.35792982, -0.26937237]),
 array([-0.36834862, -0.35792982, -0.26937237, -0.32841023]),
 array([-0.35792982, -0.26937237, -0.32841023, -0.25200771]),
 array([-0.26937237, -0.32841023, -0.25200771, -0.31104623]),
 array([-0.32841023, -0.25200771, -0.31104623, -0.25895358]),
 array([-0.25200771, -0.31104623, -0.25895358, -0.35272075]),
 array([-0.31104623, -0.25895358, -0.35272075, -0.19470599]),
 array([-0.25895358, -0.35272075, -0.19470599, -1.12890285]),
 array([